In [1]:
import netgen.gui
from ngsolve import *
%gui tk

In [2]:
from netgen.geom2d import *
geom = SplineGeometry()
pnts = [(0,0), (2,0), (2,1), (1,1), (1,2), (0,2)]
nums = [geom.AppendPoint(*p) for p in pnts]
lines = [(nums[0], nums[1], 10, 1, 0, "b"),
         (nums[1], nums[2], 10, 1, 0, "rb"),
         (nums[2], nums[3], 10, 1, 0, "ul"),
         (nums[3], nums[4], 10, 1, 0, "ru"),
         (nums[4], nums[5], 10, 1, 0, "ul"),
         (nums[5], nums[0], 10, 1, 0, "l")]

for p0, p1, bn, ml, mr, name in lines:
    geom.Append( [ "line", p0, p1 ],  bc=bn, leftdomain=ml, rightdomain=mr)
    #geom.Append( [ "line", p0, p1 ], bc=bn, leftdomain=ml, rightdomain=mr)

mesh = Mesh(geom.GenerateMesh(maxh=0.2))
Draw(mesh)

In [3]:
#def FEmSpace
fes = H1(mesh, order=3, dirichlet =".*")
a = BilinearForm(fes)
f = LinearForm(fes)
gfu = GridFunction(fes)

In [4]:
fes.ndof
print(mesh.GetBoundaries())

('default', 'default', 'default', 'default', 'default', 'default', 'default', 'default', 'default', 'default')


In [5]:
#def test and trial-functions
u = fes.TrialFunction()
v = fes.TestFunction()
a += SymbolicBFI (grad(u)*grad(v))
funcf = 1
f += SymbolicLFI (funcf*v)

In [6]:
a.Assemble()
f.Assemble()

In [7]:
gfu.vec.data = a.mat.Inverse(freedofs = fes.FreeDofs()) * f.vec
Draw (gfu, mesh, "temperature")
Draw (-grad(gfu), mesh, "grad_temp")

In [8]:
error_L2 = []
error_H1 = []
for i in [1,8]:
    fes = H1(mesh, order=i, dirichlet =".*")
    a = BilinearForm(fes)
    f = LinearForm(fes)
    gfu1 = GridFunction(fes)
    u = fes.TrialFunction()
    v = fes.TestFunction()
    a += SymbolicBFI (grad(u)*grad(v))
    funcf = 1
    f += SymbolicLFI (funcf*v)
    a.Assemble()
    f.Assemble()
    gfu1.vec.data = a.mat.Inverse(freedofs = fes.FreeDofs()) * f.vec
    
    fes = H1(mesh, order=i, dirichlet =".*")
    a = BilinearForm(fes)
    f = LinearForm(fes)
    gfu2 = GridFunction(fes)
    u = fes.TrialFunction()
    v = fes.TestFunction()
    a += SymbolicBFI (grad(u)*grad(v))
    funcf = 1
    f += SymbolicLFI (funcf*v)
    a.Assemble()
    f.Assemble()
    gfu2.vec.data = a.mat.Inverse(freedofs = fes.FreeDofs()) * f.vec
    error_L2.append(Integrate((gfu1-gfu2)*(gfu1-gfu2), mesh))
    error_H1.append(Integrate((grad(gfu1)-grad(gfu2))*(grad(gfu1)-grad(gfu2)), mesh))